In [ ]:
# To show prompt asking for uploading the requirements.txt from disk
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install GPUtil

In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch_pretrained_bert

In [19]:
import json, sys, regex
import torch
import GPUtil
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
##----------------------------------------------------
from transformers import *
from transformers import XLMRobertaConfig
from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset



In [20]:
dataset_path = "/content/drive/MyDrive/GUC/Semester 10/NLP/Project/Datasets"

In [ ]:
# Load the MARBERT tokenizer
MARBERT_tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")

# Load the MARBERT model for multi-label classification
MARBERT_model = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=11)

In [68]:
train_file = dataset_path + "/processed_data_train.xlsx"
dataset_train_raw = pd.read_excel(train_file)

In [69]:
# Preprocess your dataset
texts_train = dataset_train_raw["Tweet"].tolist()

encoded_inputs_train = MARBERT_tokenizer(texts_train, padding=True, truncation=True, return_tensors="pt")
labels_train = dataset_train_raw.iloc[:, 2:].values  # Labels start from the third column

In [70]:
# Convert the labels to tensors
labels_train = torch.tensor(labels_train, dtype=torch.float)

# Create a TensorDataset from the encoded inputs and labels
dataset_train = TensorDataset(encoded_inputs_train.input_ids, encoded_inputs_train.attention_mask, labels_train)


In [79]:
print("Length of dataset_train:", len(dataset_train))
print("Shape of input_ids tensor in dataset_train:", dataset_train.tensors[0].shape)
print("Shape of attention_mask tensor in dataset_train:", dataset_train.tensors[1].shape)
print("Shape of labels tensor in dataset_train:", dataset_train.tensors[2].shape)

Length of dataset_train: 2278
Shape of input_ids tensor in dataset_train: torch.Size([2278, 41])
Shape of attention_mask tensor in dataset_train: torch.Size([2278, 41])
Shape of labels tensor in dataset_train: torch.Size([2278, 11])


In [ ]:
# Define hyperparameters and optimizer
batch_size = 16
num_epochs = 10
learning_rate = 2e-5
optimizer = AdamW(MARBERT_model.parameters(), lr=learning_rate)

In [36]:
# Create a DataLoader for the training dataset
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

In [37]:
# Perform training
MARBERT_model.train()

for epoch in range(num_epochs):
    total_loss = 0
    
    for batch in dataloader_train:
        # Retrieve the batch inputs and labels
        input_ids, attention_mask, batch_labels = batch
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = MARBERT_model(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Update total loss
        total_loss += loss.item()
    
    # Print average loss for the epoch
    average_loss = total_loss / len(dataloader_train)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")


Epoch 1/10 - Average Loss: 0.4299
Epoch 2/10 - Average Loss: 0.3037
Epoch 3/10 - Average Loss: 0.2422
Epoch 4/10 - Average Loss: 0.2015
Epoch 5/10 - Average Loss: 0.1646
Epoch 6/10 - Average Loss: 0.1288
Epoch 7/10 - Average Loss: 0.1027
Epoch 8/10 - Average Loss: 0.0829
Epoch 9/10 - Average Loss: 0.0666
Epoch 10/10 - Average Loss: 0.0536


In [39]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
model_save_name = 'marbert_16_10_2e-5.pt'
path = F"/content/drive/MyDrive/GUC/Semester 10/NLP/Project/{model_save_name}" 

In [46]:
# Saving the trained model
torch.save(MARBERT_model.state_dict(), path)

In [44]:
files.download('checkpoint.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
# Load saved model
MARBERT_model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [63]:
test_file = dataset_path + "/processed_data_test.xlsx"
dataset_test_raw = pd.read_excel(train_file)

In [51]:
# Preprocess your dataset
texts_test = dataset_test_raw["Tweet"].tolist()

encoded_inputs_test = MARBERT_tokenizer(texts_test, padding=True, truncation=True, return_tensors="pt")
labels_test = dataset_test_raw.iloc[:, 2:].values  # Assuming labels start from the third column

In [52]:
# Convert the labels to tensors
labels_test = torch.tensor(labels_test, dtype=torch.float)

# Create a TensorDataset from the encoded inputs and labels
dataset_test = TensorDataset(encoded_inputs_test.input_ids, encoded_inputs_test.attention_mask, labels_test)

In [53]:
# Create a DataLoader for the test dataset

dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [54]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Lists to store predicted and true labels
all_predictions = []
all_true_labels = []

# Disable gradient calculation
with torch.no_grad():
    # Set the model to evaluation mode
    MARBERT_model.eval()

    # Iterate over the test dataloader
    for batch in dataloader_test:
        # Retrieve the batch inputs and labels
        input_ids, attention_mask, batch_labels = batch

        # Move the batch tensors to the device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        batch_labels = batch_labels.to(device)

        # Forward pass
        outputs = MARBERT_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Convert logits to probabilities
        probabilities = torch.sigmoid(logits)

        # Convert probabilities to predicted labels (0 or 1)
        predictions = torch.round(probabilities)

        # Append predictions and true labels to the lists
        all_predictions.extend(predictions.cpu().numpy())
        all_true_labels.extend(batch_labels.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(all_true_labels, all_predictions)
precision = precision_score(all_true_labels, all_predictions, average='weighted')
recall = recall_score(all_true_labels, all_predictions, average='weighted')
f1 = f1_score(all_true_labels, all_predictions, average='weighted')

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Accuracy: 0.9508
Precision: 0.9960
Recall: 0.9783
F1-Score: 0.9851


In [67]:
# Looping on all test sentences and showing the output probabilities.
# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the output file path
output_file = "predicted_probabilities.txt"

test_sentences = dataset_test_raw["Tweet"].tolist()

# Open the output file in write mode
with open(output_file, "w") as file:
    for sentence in test_sentences:
        file.write("******************************************************************\n")
        file.write(f"________________{sentence}____________________\n")

        # Tokenize the sentence
        encoded_input = MARBERT_tokenizer.encode_plus(
            sentence,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        # Move the input tensors to the device
        input_ids = encoded_input["input_ids"].to(device)
        attention_mask = encoded_input["attention_mask"].to(device)

        # Set the model to evaluation mode
        MARBERT_model.eval()

        # Move the model to the device
        MARBERT_model.to(device)

        # Disable gradient calculation
        with torch.no_grad():
            # Forward pass
            outputs = MARBERT_model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Convert logits to probabilities
        probabilities = torch.sigmoid(logits)

        label_names = ["anger","anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust"]  

        # Write the predicted probabilities for each label to the file
        for i, label_name in enumerate(label_names):
            file.write(f"{label_name}: {probabilities[0][i].item():.4f}\n")
            
